In [1]:
import pandas as pd
import time
import os
import requests
from riotwatcher import LolWatcher, ApiError
from dotenv import load_dotenv

# ==========================================
# CONFIGURACIÓN
# ==========================================
load_dotenv()

API_KEY = os.getenv("RIOT_API_KEY") 
RIOT_ID_NAME = "Stitch"
RIOT_ID_TAG = "FPMR"
REGION = "AMERICAS"      
PLATFORM = "LA2"         

ARCHIVO_MAESTRO = "raw_data_games.csv"

# 420 = SoloQ, 440 = Flex
TARGET_QUEUES = [420, 440]   

# AHORA SÍ PUEDES PONER NÚMEROS GRANDES AQUÍ
# El script hará las peticiones de 100 en 100 automáticamente.
CANTIDAD_A_REVISAR = 400      
MINUTO_CORTE = 15            

watcher = LolWatcher(API_KEY)

# ==========================================
# 1. FUNCIONES AUXILIARES
# ==========================================
def get_puuid(name, tag):
    try:
        user = watcher.account.by_riot_id(REGION, name, tag)
        return user['puuid']
    except:
        url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{name}/{tag}"
        headers = {"X-Riot-Token": API_KEY}
        try:
            r = requests.get(url, headers=headers)
            return r.json()['puuid'] if r.status_code == 200 else None
        except: return None

def get_recent_match_ids(puuid, count=100):
    """
    Obtiene los IDs recientes usando PAGINACIÓN (Bloques de 100).
    Esto evita el error 400 Bad Request.
    """
    unique_matches = set()
    print(f"--- Consultando API por historial reciente ({count} por cola)... ---")
    
    for q_type in TARGET_QUEUES:
        print(f"--> Buscando en cola: {q_type}")
        matches_found = []
        start = 0
        
        # --- LÓGICA DE PAGINACIÓN RECUPERADA ---
        while len(matches_found) < count:
            try:
                # Calculamos cuántas faltan
                remaining = count - len(matches_found)
                # Pedimos máximo 100 o el remanente
                request_amount = min(100, remaining)
                
                if request_amount <= 0: break

                batch = watcher.match.matchlist_by_puuid(
                    REGION, puuid, queue=q_type, start=start, count=request_amount
                )
                
                if not batch: break
                
                matches_found.extend(batch)
                start += len(batch)
                
                # Pausa pequeña para no saturar la API entre páginas
                time.sleep(0.5) 
                
            except ApiError as err:
                print(f"    Error consultando cola {q_type}: {err}")
                break
        
        unique_matches.update(matches_found)
        print(f"    Total encontrado en cola {q_type}: {len(matches_found)}")
            
    return list(unique_matches)

# ==========================================
# 2. PROCESAMIENTO (TUS COLUMNAS + SIN FILTROS)
# ==========================================
def process_match(match_id, my_puuid):
    try:
        # A. Datos Generales
        match_detail = watcher.match.by_id(REGION, match_id)
        game_duration_min = match_detail['info']['gameDuration'] / 60
        queue_id = match_detail['info']['queueId']
        
        # SIN FILTRO DE DURACIÓN (Procesa todo)

        # B. Identificarme
        participants = match_detail['info']['participants']
        me = next((p for p in participants if p['puuid'] == my_puuid), None)
        if not me: return None
        
        my_id = me['participantId']
        my_team = me['teamId']
        my_role = me['teamPosition']
        if not my_role: return None # Remake raro sin rol

        # C. Rival
        opponent = next(
            (p for p in participants if p['teamPosition'] == my_role and p['teamId'] != my_team),
            None
        )
        
        # --- TUS COLUMNAS ORIGINALES ---
        row_data = {
            'match_id': match_id,
            'queue_id': queue_id,
            'win': 1 if me['win'] else 0,
            'champion': me['championName'],
            'role': my_role,
            'game_duration': round(game_duration_min, 2),
            
            # Totales (Nombres originales)
            'kills_total': me['kills'],
            'deaths_total': me['deaths'],
            'assists_total': me['assists'],
            'lane_minions': me['totalMinionsKilled'],      
            'jungle_minions': me['neutralMinionsKilled'],  
            'vision_score': me['visionScore'],             
            'wards_placed': me['wardsPlaced'],
            'wards_killed': me['wardsKilled'],
            'control_wards_placed': me['detectorWardsPlaced']
        }

        # Challenges
        challenges = me.get('challenges', {})
        row_data['turret_plates'] = challenges.get('turretPlatesTaken', 0)
        row_data['vision_advantage'] = challenges.get('visionScoreAdvantageLaneOpponent', 0)
        row_data['control_ward_coverage'] = challenges.get('controlWardTimeCoverageInRiverOrEnemyHalf', 0)
        row_data['early_roam_kills'] = challenges.get('killsOnOtherLanesEarlyJungleAsLaner', 0)
        row_data['aces_before_15'] = challenges.get('acesBefore15Minutes', 0)

        # Objetivos
        teams_info = match_detail['info']['teams']
        my_team_obj = next(t for t in teams_info if t['teamId'] == my_team)
        objectives = my_team_obj['objectives']

        row_data['first_blood'] = 1 if objectives['champion']['first'] else 0
        row_data['first_tower'] = 1 if objectives['tower']['first'] else 0
        row_data['first_dragon'] = 1 if objectives['dragon']['first'] else 0
        row_data['first_rift_herald'] = 1 if objectives['riftHerald']['first'] else 0
        row_data['void_grubs'] = objectives['horde']['kills'] if 'horde' in objectives else 0

        # --- TIMELINE (MINUTO 15) ---
        try:
            timeline = watcher.match.timeline_by_match(REGION, match_id)
            frames = timeline['info']['frames']
            
            # Solo si la partida duró lo suficiente para tener min 15
            if len(frames) > MINUTO_CORTE:
                current_kills = 0; current_deaths = 0; current_assists = 0
                current_w_placed = 0; current_w_killed = 0; current_cw = 0

                for i, frame in enumerate(frames):
                    for event in frame['events']:
                        # KDA
                        if event['type'] == 'CHAMPION_KILL':
                            if event['killerId'] == my_id: current_kills += 1
                            if event['victimId'] == my_id: current_deaths += 1
                            if 'assistingParticipantIds' in event and my_id in event['assistingParticipantIds']: current_assists += 1
                        # Visión
                        if event['type'] == 'WARD_PLACED' and event['creatorId'] == my_id:
                            current_w_placed += 1
                            if event.get('wardType') == 'CONTROL_WARD': current_cw += 1
                        if event['type'] == 'WARD_KILL' and event['killerId'] == my_id:
                            current_w_killed += 1

                    # SNAPSHOT
                    if i == MINUTO_CORTE:
                        col_prefix = f"min{MINUTO_CORTE}"
                        row_data[f'{col_prefix}_kills'] = current_kills
                        row_data[f'{col_prefix}_deaths'] = current_deaths
                        row_data[f'{col_prefix}_assists'] = current_assists
                        row_data[f'{col_prefix}_wards_placed'] = current_w_placed
                        row_data[f'{col_prefix}_wards_killed'] = current_w_killed
                        row_data[f'{col_prefix}_control_wards'] = current_cw
                        
                        # Oro/CS
                        if str(my_id) in frame['participantFrames']:
                            my_frame = frame['participantFrames'][str(my_id)]
                            row_data[f'{col_prefix}_gold'] = my_frame['totalGold']
                            row_data[f'{col_prefix}_cs'] = my_frame['minionsKilled'] + my_frame['jungleMinionsKilled']
                            
                            if opponent and str(opponent['participantId']) in frame['participantFrames']:
                                opp_frame = frame['participantFrames'][str(opponent['participantId'])]
                                row_data[f'{col_prefix}_gold_diff'] = my_frame['totalGold'] - opp_frame['totalGold']
                                row_data[f'{col_prefix}_cs_diff'] = (my_frame['minionsKilled'] + my_frame['jungleMinionsKilled']) - (opp_frame['minionsKilled'] + opp_frame['jungleMinionsKilled'])
                                row_data[f'{col_prefix}_xp_diff'] = my_frame['xp'] - opp_frame['xp']
                            else:
                                row_data[f'{col_prefix}_gold_diff'] = 0; row_data[f'{col_prefix}_cs_diff'] = 0; row_data[f'{col_prefix}_xp_diff'] = 0
                        else:
                            row_data[f'{col_prefix}_gold'] = 0; row_data[f'{col_prefix}_cs'] = 0
                        break 
        except Exception:
            pass # Si falla timeline o no existe, seguimos con los datos generales

        return row_data
    except Exception: return None

# ==========================================
# 3. LÓGICA DE ACTUALIZACIÓN INCREMENTAL
# ==========================================
if __name__ == "__main__":
    puuid = get_puuid(RIOT_ID_NAME, RIOT_ID_TAG)

    if puuid:
        print(f"PUUID Encontrado. Iniciando actualización...")
        
        # 1. CARGAR DATOS ANTIGUOS
        existing_matches = set()
        df_old = pd.DataFrame()
        
        if os.path.exists(ARCHIVO_MAESTRO):
            print(f"📂 Archivo maestro: {ARCHIVO_MAESTRO}")
            try:
                df_old = pd.read_csv(ARCHIVO_MAESTRO)
                existing_matches = set(df_old['match_id'].unique())
                print(f"   --> Registros actuales: {len(df_old)}")
            except Exception as e:
                print(f"   ⚠️ Error leyendo: {e}")
        else:
            print(f"📂 Creando archivo nuevo.")

        # 2. BUSCAR NUEVAS PARTIDAS (Con Paginación Segura)
        recent_ids = get_recent_match_ids(puuid, count=CANTIDAD_A_REVISAR)
        
        # 3. FILTRAR: ¿Cuáles faltan?
        new_ids_to_process = [mid for mid in recent_ids if mid not in existing_matches]
        
        if not new_ids_to_process:
            print("\n✅ TODO AL DÍA.")
        else:
            print(f"\n🚀 {len(new_ids_to_process)} partidas NUEVAS. Procesando...")
            
            new_data_list = []
            for i, mid in enumerate(new_ids_to_process):
                print(f"   Procesando {i + 1}/{len(new_ids_to_process)}: {mid}")
                row = process_match(mid, puuid)
                if row:
                    new_data_list.append(row)
                time.sleep(1.2) 

            # 4. CONCATENAR Y GUARDAR
            if new_data_list:
                df_new = pd.DataFrame(new_data_list)
                
                # FUSIÓN
                df_final = pd.concat([df_old, df_new], ignore_index=True)
                df_final = df_final.drop_duplicates(subset=['match_id'], keep='last')
                
                df_final.to_csv(ARCHIVO_MAESTRO, index=False)
                
                print("-" * 40)
                print(f"✅ ACTUALIZACIÓN EXITOSA")
                print(f"   Agregadas: {len(df_new)}")
                print(f"   Total: {len(df_final)}")
                print("-" * 40)
            else:
                print("⚠️ No se pudo procesar ninguna partida nueva (error API o datos nulos).")
    else:
        print("❌ Error de PUUID.")

PUUID Encontrado. Iniciando actualización...
📂 Archivo maestro: raw_data_games.csv
   --> Registros actuales: 445
--- Consultando API por historial reciente (400 por cola)... ---
--> Buscando en cola: 420
    Total encontrado en cola 420: 256
--> Buscando en cola: 440
    Total encontrado en cola 440: 167

🚀 6 partidas NUEVAS. Procesando...
   Procesando 1/6: LA2_1557595062
   Procesando 2/6: LA2_1557429946
   Procesando 3/6: LA2_1557512828
   Procesando 4/6: LA2_1557598503
   Procesando 5/6: LA2_1557504125
   Procesando 6/6: LA2_1557441605
----------------------------------------
✅ ACTUALIZACIÓN EXITOSA
   Agregadas: 6
   Total: 451
----------------------------------------
